## All imports needed for detectron2

In [1]:
import detectron2
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import ColorMode
from detectron2.data import DatasetCatalog
from detectron2 import data
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
import cv2
import random
import time
import numpy as np
import torch, gc

ModuleNotFoundError: No module named 'detectron2'

## This block of code below provides you to empty cuda cache (free gpu memory), find COCO datasets and start to read all images.

In [2]:
torch.cuda.empty_cache()

## Change this to adapt the path where you have installed detectron2 framework
DETECTRON2_ROOT = "/home/user"

register_coco_instances("Speed", {}, DETECTRON2_ROOT + "/detectron2/datasets/custom_coco/annotations/new_speed-30.json", DETECTRON2_ROOT + "/detectron2/datasets/custom_coco/images")

Speed_metadata = MetadataCatalog.get("Speed")
dataset_dicts = DatasetCatalog.get("Speed")
i = 0
for d in random.sample(dataset_dicts, 1):
    i = i +1
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=Speed_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2.imshow("test" + str(i), vis.get_image()[:, :, ::-1])



Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



## This one is setting all parameters for training nothing to change except SOLVER.IMS_PER_BATCH you can if you a a good GPU go to 8.

In [3]:
cfg = get_cfg()
cfg.merge_from_file(
    "configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)

cfg.DATASETS.TRAIN = ("Speed",)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.01 * 2 / 16
cfg.SOLVER.MAX_ITER = (
    300
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # 5 classes (locomotive, container, quad, crane, wagon)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("Speed", )
predictor = DefaultPredictor(cfg)

[09/20 13:16:10 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [09/20 13:16:10 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[09/20 13:16:10 d2.data.datasets.coco]: Loaded 190 images in COCO format from /home/cristal/detectron2/datasets/custom_coco/annotations/new_speed-30.json
[09/20 13:16:10 d2.data.build]: Removed 0 images with no usable annotations. 190 images left.
[09/20 13:16:10 d2.data.build]: Distribution of instances among all 5 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| container  | 211          |   train    | 18           |  portique  | 58           |
|   TQuad    | 60           |   wagon    | 4            |            |              |
|   total    | 351          |            |              |            |              |
[09/20 13:16:10 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShorte

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (6, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (6,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (20, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (20,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (5, 256, 1, 1) 

[09/20 13:16:11 d2.engine.train_loop]: Starting training from iteration 0
ERROR [09/20 13:16:11 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/home/cristal/detectron2/detectron2/engine/train_loop.py", line 149, in train
    self.run_step()
  File "/home/cristal/detectron2/detectron2/engine/defaults.py", line 499, in run_step
    self._trainer.run_step()
  File "/home/cristal/detectron2/detectron2/engine/train_loop.py", line 273, in run_step
    loss_dict = self.model(data)
  File "/home/cristal/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/cristal/detectron2/detectron2/modeling/meta_arch/rcnn.py", line 154, in forward
    features = self.backbone(images.tensor)
  File "/home/cristal/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/cristal/detectron

/home/cristal/.local/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/home/cristal/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 3.95 GiB total capacity; 872.13 MiB already allocated; 4.25 MiB free; 904.00 MiB reserved in total by PyTorch)

## This last block show us randoms images from the dataset with detected object of the classes.

In [ ]:
for d in random.sample(dataset_dicts, 3): 
    i = i +1   
    im = cv2.imread(d["file_name"])
    imS = cv2.resize(im,(1920, 1080))
    outputs = predictor(imS)
    v = Visualizer(imS[:, :, ::-1],
                   metadata=Speed_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("test" + str(i), v.get_image()[:, :, ::-1])

times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(imS)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

#(this is necessary to avoid Python kernel form crashing) 
cv2.waitKey(0)  
#closing all open windows  
cv2.destroyAllWindows()